In [1]:
import cv2
import numpy as np
import requests
from twilio.rest import Client 
import time
from tkinter import Tk
from urllib.request import Request, urlopen
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
def timediff(t1,t2):
    diff=int(t2[17:19])-int(t1[17:19])
    if diff < 0:
        return diff+60
    else:
        return diff
account_sid = 'AC58887497f62b22bbf5393a28b8481132'     #provided by Twilio
auth_token = '52727ec69031a3354aac9a78494faf33' 
client = Client(account_sid, auth_token) 
trial=0
diff=31
message1='Someone on your gate at:'                    # Enter customised notification message
url="http://192.168.2.5/cgi-bin/snapshot.cgi?chn=3&u=admin&p=&q=0&d=1&rand=0.048224097639987695"   #CCTV input link
img_resp=requests.get(url)
img_arr=np.array(bytearray(img_resp.content),dtype=np.uint8)
frame1=cv2.imdecode(img_arr,-1)
img_resp=requests.get(url)
img_arr=np.array(bytearray(img_resp.content),dtype=np.uint8)
frame2=cv2.imdecode(img_arr,-1)
t1=str(datetime.now())
while True:
    diff = cv2.absdiff(frame1,frame2)
    gray=cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    _,thresh=cv2.threshold(blur,20,255,cv2.THRESH_BINARY)
    dilated=cv2.dilate(thresh,None,iterations=3)
    contours=[]
    contours,_=cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if cv2.contourArea(contour)<5000:
        cv2.putText(frame1,"Status:{}".format('static'),(10,20),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),3)
    for contour in contours:
        (x,y,w,h)=cv2.boundingRect(contour)
        t2=str(datetime.now())
        diff=timediff(t1,t2)
        if cv2.contourArea(contour)<5000 or diff<16:    
            continue
        time.sleep(2)
        cv2.imwrite('desktop/xyz.jpg',frame2)
        driver = webdriver.Chrome(ChromeDriverManager().install())
        web='https://www.file.io/'
        driver.get(web)
        xpath='//*[@id="fine-uploader"]/div/div/input'
        driver.find_element_by_xpath(xpath).send_keys('C:\\Users/Shravan Tawri/Desktop/xyz.jpg')
        time.sleep(4)
        k=driver.find_element_by_xpath('//*[@id="fine-uploader"]/div/ul/li/span/button[1]')
        time.sleep(4)
        k.click()
        message = client.messages.create( 
                              from_='whatsapp:+14155238886',  
                              body=message1 + " " + str(datetime.now()),
                              media_url=Tk().clipboard_get(),
                              to='whatsapp:+9194051842**' #Number for whatsapp message
                          )
        t1=str(datetime.now())
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.putText(frame1,"Status:{}".format('Movement'),(10,20),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
        trial+=1
    cv2.imshow('feed',frame1)
    frame1=frame2
    img_resp=requests.get(url)
    img_arr=np.array(bytearray(img_resp.content),dtype=np.uint8)
    frame2=cv2.imdecode(img_arr,-1)
    if cv2.waitKey(40) & 0xFF == 27:
        break
cv2.destroyAllWindows() 